In [ ]:
# General imports
import os
import glob
import healpy
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.io.fits.header import Header
from astropy.table import Table, Column, vstack

# MER_DataModelUtils imports
from MER_DataModelUtils import DmUtils as dm_utils
from MER_DataModelUtils import MerDmUtils as mer_utils

# MER_PsfMosaic imports
from MER_PsfMosaic.ArrayUtils import ArrayUtils
from MER_PsfMosaic.EuclidWcs import EuclidWcs

# Make the plots interactive
%matplotlib notebook

In [ ]:
def get_tile_wcs(tile):
    # Get a dictionary with the tile WCS information
    wcs_dict = tile.get_wcs()

    # Correct the CRPIX values to coincide with those used by Swarp
    wcs_dict["CRPIX1"]["value"] = (wcs_dict["NAXIS1"]["value"] + 1)/2
    wcs_dict["CRPIX2"]["value"] = (wcs_dict["NAXIS2"]["value"] + 1)/2

    # Create a header object with the wcs information
    header = Header()

    for key, val in wcs_dict.items():
        header.set(key, val["value"], val["comment"])

    # Return the tile Wcs
    return EuclidWcs(header)

def add_pixel_coordinates(catalog, wcs, ra_column, dec_column):
    """ Adds the pixel coordinates information to the given catalog.

    """
    # Get the (RA, Dec) coordinates
    radec_coord = np.array([catalog[ra_column], catalog[dec_column]]).T

    # Calculate the (x, y) pixel coordinates
    xy_coord = wcs.calculate_pixel_coordinates(radec_coord)

    # Add or update the relevant columns
    if "MOSAIC_X" in catalog.colnames:
        catalog["MOSAIC_X"] = xy_coord[:, 0]
        catalog["MOSAIC_Y"] = xy_coord[:, 1]
    else:
        catalog.add_column(Column(name="MOSAIC_X", data=xy_coord[:, 0]))
        catalog.add_column(Column(name="MOSAIC_Y", data=xy_coord[:, 1]))

def select_true_universe_sources(catalog_file_names, ra_range, dec_range):
    """ Loads all the True Universe catalog files and selects only those
    sources that fall inside the specified (RA, Dec) region.

    """
    # Loop over the True Universe catalog files and select the relevant sources
    merged_catalog = None

    for file_name in catalog_file_names:
        # Load the catalog table
        catalog = Table.read(file_name, format="fits")

        # Get the (RA, Dec) columns
        ra = catalog["ra_mag"] if "ra_mag" in catalog.colnames else catalog["RA"]
        dec = catalog["dec_mag"] if "dec_mag" in catalog.colnames else catalog["DEC"]

        # Check which sources fall inside the given (RA, Dec) ranges
        cond_ra = np.logical_and(ra > ra_range[0], ra < ra_range[1])
        cond_dec = np.logical_and(dec > dec_range[0], dec < dec_range[1])
        cond = np.logical_and(cond_ra, cond_dec)

        if np.any(cond):
            # Add the selected sources to the merged catalog
            if merged_catalog is None:
                merged_catalog = catalog[cond]
            else:
                merged_catalog = vstack([merged_catalog, catalog[cond]])

    return merged_catalog

def select_tile_true_universe_sources(catalog_file_names, tile):
    """ Loads all the True Universe catalog files and selects only those
    sources that fall inside the tile inner area.

    """
    # Get the tile healpix indices
    tile_healpix = tile.get_inner_healpix_indices()[1]

    # Get the tile MOC order
    moc_order = tile.get_tile_data().MocOrder

    # Calculate the tile nside parameter
    nside = healpy.order2nside(moc_order)

    # Loop over the True Universe catalog files and select the relevant sources
    merged_catalog = None

    for file_name in catalog_file_names:
        # Load the catalog table
        catalog = Table.read(file_name, format="fits")

        # Get the (RA, Dec) columns
        ra = catalog["ra_mag"] if "ra_mag" in catalog.colnames else catalog["RA"]
        dec = catalog["dec_mag"] if "dec_mag" in catalog.colnames else catalog["DEC"]

        # Convert the units from degrees to radians
        theta = np.pi / 2 - np.pi * dec / 180
        phi = np.pi * ra / 180

        # Get the healpix indices
        sources_healpix = healpy.pixelfunc.ang2pix(nside, theta, phi, nest=True)

        # Check which sources fall inside the tile
        cond = np.in1d(sources_healpix, tile_healpix, assume_unique=False)

        if np.any(cond):
            # Add the selected sources to the merged catalog
            if merged_catalog is None:
                merged_catalog = catalog[cond]
            else:
                merged_catalog = vstack([merged_catalog, catalog[cond]])

    return merged_catalog

In [ ]:
# Get the True Universe star and galaxy catalog file names
star_catalog_file_names = glob.glob("/home/jgracia/Work/TU/stars/v7/*.fits")
galaxy_catalog_file_names = glob.glob("/home/jgracia/Work/TU/galaxies/v8.1/*.fits")

In [ ]:
# Set the field number and the tile number
field_number = 52929
tile_number = 40012

# Set the path to the tile directory
tiledir = "/home/jgracia/Work/sc456Tiles/field" + str(field_number)

# Add some extra functionality to the MER data product bindings
mer_utils.init()

# Load the tile xml
tile_xml = dm_utils.read_tile_metadata(os.path.join(tiledir,"tile%s.xml" % tile_number))

# Get the tile Wcs
wcs = get_tile_wcs(tile_xml)

# Calculate the range of RA and Dec values covered by the tile
xy_corners = np.array([[0, 0], wcs.get_naxis()]) + 0.5
radec_corners = wcs.calculate_world_coordinates(xy_corners)
ra_range = np.sort(radec_corners[:, 0])
dec_range = np.sort(radec_corners[:, 1])

In [ ]:
# Select only those TU sources that fall inside the tile
#star_catalog = select_true_universe_sources(star_catalog_file_names, ra_range, dec_range)
#galaxy_catalog = select_true_universe_sources(galaxy_catalog_file_names, ra_range, dec_range)
star_catalog = select_tile_true_universe_sources(star_catalog_file_names, tile_xml)
galaxy_catalog = select_tile_true_universe_sources(galaxy_catalog_file_names, tile_xml)

# Add the (x, y) pixel coordinate information to the catalogs
add_pixel_coordinates(star_catalog, wcs, ra_column="RA", dec_column="DEC")
add_pixel_coordinates(galaxy_catalog, wcs, ra_column="ra_mag", dec_column="dec_mag")

# Save the True Universe catalogs in the tile directory
star_catalog.write(os.path.join(tiledir, "tu_star_catalog_v7_tile%s.fits" % tile_number), format="fits", overwrite=True)
galaxy_catalog.write(os.path.join(tiledir, "tu_galaxy_catalog_v8p1_tile%s.fits" % tile_number), format="fits", overwrite=True)

In [ ]:
# Set the path to the working directory
workdir = "/home/jgracia/Work/pre-release-07-12-2018"

# Load the MER xml products
mosaic_xml = dm_utils.read_product_metadata(os.path.join(workdir,
                "euclidBandsOperations.iterations.1.MER_Background/background.xml"))
catalog_xml = dm_utils.read_product_metadata(os.path.join(workdir,
                "MER_Catalog/final_catalog.xml"))

In [ ]:
# Load the mosaic image data
mosaic_file_name = os.path.join(workdir, "data", mosaic_xml.get_data())
mosaic_data, header = fits.getdata(mosaic_file_name, 0, header=True)

# Get a Euclid WCS object from the header information
mosaic_wcs = EuclidWcs(header)

In [ ]:
# Load the MER final catalog
catalog_file_name = os.path.join(workdir, "data", catalog_xml.get_data())
catalog = Table.read(catalog_file_name, format="fits")

# Add the (x, y) pixel coordinate information to the catalog
add_pixel_coordinates(catalog, mosaic_wcs, ra_column="RIGHT_ASCENSION", dec_column="DECLINATION")

In [ ]:
# Update the TU (x, y) pixel coordinates that they refer to the mosaic wcs
add_pixel_coordinates(star_catalog, mosaic_wcs, ra_column="RA", dec_column="DEC")
add_pixel_coordinates(galaxy_catalog, mosaic_wcs, ra_column="ra_mag", dec_column="dec_mag")

In [ ]:
# Estimate the signal range that should be used to display the mosaic
central_region = ArrayUtils.get_region(mosaic_data, center=np.array(mosaic_data.shape) / 2, size=1001)
vmin = np.percentile(central_region, 0.05)
vmax = np.percentile(central_region, 99.65)

# Select the MER final catalog sources that we want to plot
to_plot = catalog["STAR_FLAG"] == 1

# Plot the True Universe star positions and the MER detected sources on top of the mosaic image
plt.figure()
plt.imshow(mosaic_data, origin="lower", vmin=vmin, vmax=vmax)
plt.colorbar()
plt.gca().set_xlim(auto=False)
plt.gca().set_ylim(auto=False)
plt.plot(star_catalog["MOSAIC_X"] - 1, star_catalog["MOSAIC_Y"] - 1, "r.", markersize=4)
plt.plot(catalog["MOSAIC_X"][to_plot] - 1, catalog["MOSAIC_Y"][to_plot] - 1, "ro", markersize=7, fillstyle="none")
plt.title("%s mosaic" % mosaic_xml.get_filter())
plt.xlabel("x (pixels)")
plt.ylabel("y (pixels)")
plt.show()